In [38]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [39]:
print (mnist.train.images.shape)

(55000, 784)


In [42]:
print(mnist.train)

# Loading data minist

###  we might have guessed 60000 represents the number of images in the train dataset and (28, 28) represents the size of the image: 28 x 28 pixel.

In [3]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
import matplotlib.pyplot as plt
#x_train and x_test parts contain greyscale RGB codes (from 0 to 255) 
#while y_train and y_test parts contains labels from 0 to 9 which represents which number they actually are. 
image_index = 7777 # select anything up to 60,000
print(y_train[image_index]) # The label is 8
plt.imshow(x_train[image_index], cmap='Greys')
# we will get the greyscale visualization of the RGB codes as shown below.

8


In [5]:
x_train.shape 
#we might have guessed 60000 represents the number of images in the train dataset and (28, 28) represents the size of the image: 28 x 28 pixel.

(60000, 28, 28)

# Support Vector Classifier

In [6]:
from sklearn.svm import SVC # "Support vector classifier"
import numpy as np
from sklearn.metrics import accuracy_score

In [7]:
X_train = np.reshape(x_train,(60000, -1))

In [8]:
X_train.shape

(60000, 784)

In [9]:
x_test.shape
X_test = np.reshape(x_test,(10000, -1))
X_test.shape

(10000, 784)

In [10]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [11]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

### We use the number '0' for example

In [11]:
Y_train0 = y_train.copy()
Y_test0 = y_test.copy()

Y_train0[Y_train0!=0] = -1
Y_train0[Y_train0==0] = 1
Y_test0[Y_test0!=0] = -1
Y_test0[Y_test0==0] = 1

### We start by summarizing the results of the virtual support vector method. We trained ten binary classifiers using C =10. We used a polynomial kernel K(x, y) = (x· y)5.

In [43]:
svmpoly0 = SVC(C = 10, kernel = 'poly', degree=5, gamma ='auto')
svmpoly0.fit(X_train, Y_train0)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=5, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [44]:
svmpoly0.support_vectors_.shape

(1760, 784)

In [45]:
sv0 = svmpoly0.support_vectors_  #Support vectors.
print("Nsv ORIG", sv0.shape[0])

Nsv ORIG 1760


In [15]:
sv0_index = svmpoly0.support_     #Number of support vectors for each class.

In [16]:
sv0_index

array([   18,    80,   158, ..., 59653, 59681, 59934], dtype=int32)

In [17]:
sv0_label = Y_train0[sv0_index]

In [18]:
y_pred0 = svmpoly0.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test0, y_pred0))

0.9958


In [28]:
print("NE ORIG",len(Y_test0[Y_test0!=y_pred0]))

42


# One pixel in 4 direction pan

### We then generated new training data by translating the resulting support vectors by one pixel in each of four directions, and trained a new machine (using the same parameters). 

In [29]:
sv0_28 = np.reshape(sv0,(-1, 28, 28))

In [30]:
sv0_newR = np.zeros((1760,28, 28))
sv0_newR[:, :, 1:28]= sv0_28[:, :, 0:27]
trans_R = sv0_newR

In [31]:
sv0_newL = np.zeros((1760,28, 28))
sv0_newL[:, :, 0:27]= sv0_28[:, :, 1:28]
trans_L = sv0_newL

In [32]:
sv0_newD = np.zeros((1760,28, 28))
sv0_newD[:, 1:28, :]= sv0_28[:, 0:27, :]
trans_D = sv0_newD

In [33]:
sv0_newU = np.zeros((1760,28, 28))
sv0_newU[:, 0:27, :]= sv0_28[:, 1:28, :]
trans_U = sv0_newU

In [34]:
sv0_new_con = np.concatenate((trans_R,trans_L,trans_D,trans_U),axis=0)

In [35]:
new_y_train0 = np.tile(sv0_label,4)

### Train a new machine by using the same parameters

In [36]:
sv0_new_con = np.reshape(sv0_new_con,(7040, -1))

In [37]:
svmpoly0 = SVC(C=10, kernel = 'poly', gamma ='auto', degree=5 )
#svmpoly0 = SVC(kernel = 'poly', gamma ='auto' )  Higher accuracy when 'C' and 'degree' are not specified
svmpoly0.fit(sv0_new_con, new_y_train0)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=5, gamma='auto', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
VSV = svmpoly0.support_vectors_
print("Nsv VSV",VSV.shape[0])

In [38]:
y_pred0_new = svmpoly0.predict(X_test)

In [40]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test0, y_pred0_new))
print("NE VSV",len(Y_test0[Y_test0!=y_pred0_new]))

0.9976
NE ORIG 24


## For the number from 0 to 9

In [12]:
for i in range(0,10,1):
    print("Digital is ",i)
    Y_train = y_train.copy()
    Y_test = y_test.copy()

    Y_train[Y_train!=i] = -1
    Y_train[Y_train==i] = 1
    Y_test[Y_test!=i] = -1
    Y_test[Y_test==i] = 1
    
    svmpoly = SVC(C = 10, kernel = 'poly', degree=5, gamma ='auto')
    svmpoly.fit(X_train, Y_train)
    
    sv = svmpoly.support_vectors_ 
    print("Nsv ORIG for digital",i, "is", sv.shape[0])
    
    y_pred = svmpoly.predict(X_test)
    print("ORIG accuracy_score: ",accuracy_score(Y_test, y_pred))
    print("NE ORIG for digital",i , "is: ", len(Y_test[Y_test!=y_pred]))
    
    sv_index = svmpoly.support_
    sv_label = Y_train[sv_index]
    
    sv_28 = np.reshape(sv,(-1, 28, 28))
    
    sv_newR = np.zeros((sv.shape[0],28, 28))
    sv_newR[:, :, 1:28]= sv_28[:, :, 0:27]
    trans_R = sv_newR
    
    sv_newL = np.zeros((sv.shape[0],28, 28))
    sv_newL[:, :, 0:27]= sv_28[:, :, 1:28]
    trans_L = sv_newL
    
    sv_newD = np.zeros((sv.shape[0],28, 28))
    sv_newD[:, 1:28, :]= sv_28[:, 0:27, :]
    trans_D = sv_newD
    
    sv_newU = np.zeros((sv.shape[0],28, 28))
    sv_newU[:, 0:27, :]= sv_28[:, 1:28, :]
    trans_U = sv_newU
    
    sv_new_con = np.concatenate((trans_R,trans_L,trans_D,trans_U),axis=0)
    new_y_train = np.tile(sv_label,4)
    
    sv_new_con = np.reshape(sv_new_con,(sv.shape[0]*4, -1))
    svmpoly = SVC(C=10, kernel = 'poly', gamma ='auto', degree=5 )
    #svmpoly = SVC(kernel = 'poly', gamma ='auto' )  Higher accuracy when 'C' and 'degree' are not specified
    svmpoly.fit(sv_new_con, new_y_train)
    vsv = svmpoly.support_vectors_ 
    print("Nsv VSV for digital",i, "is: ", vsv.shape[0]) 
    
    y_pred_new = svmpoly.predict(X_test)
    print("VSV accuracy_score: ",accuracy_score(Y_test, y_pred_new))
    print("NE VSV for digital",i , "is: ", len(Y_test[Y_test!=y_pred_new]))

Digital is  0
Nsv ORIG for digital 0 is 1760
ORIG accuracy_score:  0.9958
NE ORIG for digital 0 is:  42
Nsv VSV for digital 0 is:  4004
VSV accuracy_score:  0.9976
NE VSV for digital 0 is:  24
Digital is  1
Nsv ORIG for digital 1 is 1393
ORIG accuracy_score:  0.9959
NE ORIG for digital 1 is:  41
Nsv VSV for digital 1 is:  3067
VSV accuracy_score:  0.9967
NE VSV for digital 1 is:  33
Digital is  2
Nsv ORIG for digital 2 is 2856
ORIG accuracy_score:  0.9923
NE ORIG for digital 2 is:  77
Nsv VSV for digital 2 is:  6465
VSV accuracy_score:  0.9948
NE VSV for digital 2 is:  52
Digital is  3
Nsv ORIG for digital 3 is 3159
ORIG accuracy_score:  0.9928
NE ORIG for digital 3 is:  72
Nsv VSV for digital 3 is:  7104
VSV accuracy_score:  0.9954
NE VSV for digital 3 is:  46
Digital is  4
Nsv ORIG for digital 4 is 2431
ORIG accuracy_score:  0.9927
NE ORIG for digital 4 is:  73
Nsv VSV for digital 4 is:  5472
VSV accuracy_score:  0.9954
NE VSV for digital 4 is:  46
Digital is  5
Nsv ORIG for digital 